# Modelo Predicción de Ocupación

## Contenido
* Parámetros generales e Importacion de biblioticas
* Carga de los datos
* Parámetros Generales
* Preprocesamiento
 * Unificación 
* Modelo
 * Procesamiento
 * Tidy Table
 * Separación Xvar y Yvar  (número variable de periodo)
 * Separacion Train y Test
 * Configuración de los Modelos
 * Funciones 
  * F. para evaluar varios modelos
  * F. para categorizar en semáforo
  * F. Evaluación de resultados Exactitud
* Generacion de Pronosticos a futuro
 * Procesamiento entrada para pronóstico
 * Ejecución del modelos y archivo de entrega

## Parámetros generales e Importacion de biblioticas

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from datetime import datetime
from dateutil.relativedelta import relativedelta
from imblearn.over_sampling import SMOTE
import xgboost as xgb
from xgboost import plot_importance
from lightgbm import LGBMClassifier
from sklearn.metrics import confusion_matrix # es una manera para resumir los datos 
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve #roc_curve
from sklearn.metrics import roc_auc_score #auc

## Carga de los datos

In [3]:
df_Nodosokla= pd.read_csv('Modelo xgboost and LGBM/datosNuevoModelo/NODOS_INFO_OOKLA.csv',sep=';', header = 0)
data_ocupacionNodo = pd.read_csv('Modelo xgboost and LGBM/datosNuevoModelo/OCUPACION_NODO.csv', sep=';')
data_NodoINFI = pd.read_csv('Modelo xgboost and LGBM/datosNuevoModelo/NODO_INFI.csv', sep=';')
data_SEG = pd.read_csv('Modelo xgboost and LGBM/datosNuevoModelo/compiladoFinal.csv', sep=';') 

### Base OOKLA

In [4]:
df_Nodosokla['DIA']=1 # le agrego la columna dia
df_Nodosokla['MES'] = [f'0{x}' if x <10 else str(x) for x in df_Nodosokla['MES']]
df_Nodosokla['DIA'] = [f'0{x}' if x <10 else str(x) for x in df_Nodosokla['DIA']]
df_Nodosokla['FECHA'] = df_Nodosokla['ANIO'].astype('str') + '-' + df_Nodosokla['MES'] + '-' + df_Nodosokla['DIA']
# eliminamos columnas cobrantes
df_Nodosokla=df_Nodosokla.drop(['DIA','MES','ANIO'], axis=1)
# ordenamos
df_Nodosokla = df_Nodosokla[['NODO','FECHA', 'OPERADOR','CANTIDAD','DOWNLOAD_KBPS','UPLOAD_KBPS','LATENCY','FLAG_LAT']] 
# # renombramos columnas
df_Nodosokla.rename(columns={'NODO':'COD_NODO'}, inplace=True)
df_Nodosokla_Claro=df_Nodosokla[df_Nodosokla["OPERADOR"]=="CLARO"]
del(df_Nodosokla_Claro['OPERADOR'])

### Base data_NodoINF

In [5]:
data_NodoINFI['DIA']=1 # le agrego la columna dia
data_NodoINFI['MES'] = [f'0{x}' if x <10 else str(x) for x in data_NodoINFI['MES']]
data_NodoINFI['DIA'] = [f'0{x}' if x <10 else str(x) for x in data_NodoINFI['DIA']]
data_NodoINFI['FECHA'] = (2000+data_NodoINFI['ANIO']).astype('str') + '-' + data_NodoINFI['MES'] + '-' + data_NodoINFI['DIA']
# eliminamos columnas sobrantes
data_NodoINFI=data_NodoINFI.drop(['DIA','MES','ANIO'], axis=1)
# # renombramos columnas
data_NodoINFI.rename(columns={'NODO':'COD_NODO'}, inplace=True)
### completar Nullos con 0
data_NodoINFI=data_NodoINFI.fillna(0)

### Base data_NodoINF data_SEG

In [6]:
data_SEG.drop(['NUEVA ESTRUCTURA'], axis=1,inplace=True)
data_SEG['SEGMENTADO']=1

## Preprocesamiento
### Unificacion

In [7]:
data_ocupacion=pd.merge(data_ocupacionNodo, data_NodoINFI, on = ('COD_NODO','FECHA'), how='left')
#data_ocupacion=pd.merge(data_ocupacion, df_Nodosokla_Claro, on = ('COD_NODO','FECHA'), how='left')
data_ocupacion=pd.merge(data_ocupacion, data_SEG, on = ('COD_NODO','FECHA'), how='left')

### Parámetros generales

In [8]:
fechaMaxima=datetime.strptime(data_ocupacion['FECHA'].max(),'%Y-%m-%d')
periodos_y=7
periodos_x=7
variable_prediccion='OCUPACION_DW'

In [9]:
fechaLimite=fechaMaxima + relativedelta(months=-1*periodos_y)
fechaInicial=fechaLimite + relativedelta(months=-1*periodos_x)

### Selección Fechas

In [10]:
## Eliminar informaición anterior Junion 2022
data_ocupacion['FECHA']=data_ocupacion['FECHA'].astype('datetime64[ns]')
### Elininar datos últimos mes
data_ocupacion=data_ocupacion[data_ocupacion['FECHA']>=fechaInicial]
### Eliminar Fecha Final  (mes incompleto)
data_ocupacion=data_ocupacion[data_ocupacion['FECHA']<fechaMaxima]

##  Funciones

In [11]:
#Funcion para Calcular el número de periodos hacia atrás
def diff_month(d1, d2):
    return (d1.year - d2.dt.year) * 12 + d1.month - d2.dt.month

In [12]:
def semaforo(df, col):
    df = df.copy()
    df.loc[(df[col]>=0.6) & (df[col]<=0.8), 'ESTATUS'] = 'AMARILLO'
    df.loc[df[col]>0.8, 'ESTATUS'] = 'ROJO'
    df.loc[df[col]<0.6, 'ESTATUS'] = 'VERDE'
    return(df)

In [13]:
def semaforoVal(val):
    #df = df.copy()
    respuesta='V'
    if (val>=0.6) & (val<=0.8):
        respuesta='A'
    elif val>0.8:
        respuesta='R'
    return(respuesta)

In [14]:
def semaforoNum2Val(val):   
    respuesta='V'
    if val==1:
        respuesta='A'
    elif val==2:
        respuesta='R'
    return(respuesta)

In [15]:
def semaforoRojo(val):    
    if val>0.8:
        respuesta=1
    else :
        respuesta=0
    return(respuesta)

In [16]:
def semaforoAmarillo(val):    
    if (val>=0.6) & (val<=0.8):
        respuesta=1
    else :
        respuesta=0
    return(respuesta)

In [17]:
def semaforoRojoAmarillo(val):    
    if val>=0.6:
        respuesta=1
    else :
        respuesta=0
    return(respuesta)

In [18]:
def semaforoNum(val):
    #df = df.copy()
    respuesta=0
    if (val>=0.6) & (val<=0.8):
        respuesta=1
    elif val>0.8:
        respuesta=2
    return(respuesta)

In [19]:
def semaforoNum2Rojo(val):    
    respuesta=0
    if val==2:
        respuesta=1    
    return(respuesta)

In [20]:
def semaforoNum2Amarillo(val):    
    respuesta=0
    if val==1:
        respuesta=1    
    return(respuesta)

In [21]:
def semaforoNum2RojoAmarillo(val):    
    respuesta=0
    if val>=1:
        respuesta=1    
    return(respuesta)

In [22]:
def medidasEvaluacion(matrizConfucion):
    exactitud=np.trace(matrizConfucion)/matrizConfucion.sum()
    print ('exactitud:', exactitud)
    sensibilidad=[0]*3
    especificidad=[0]*3
    for i in range(0,len(matrizConfucion)):
        sensibilidad[i]=matrizConfucion[i,i]/matrizConfucion.sum(axis=1)[i]
        especificidad[i]=np.delete(np.delete(matrizConfucion,i, axis=0),i,axis=1).sum()/np.delete(matrizConfucion,i, axis=0).sum()
        print ('sensibilidad ',i,": ",sensibilidad[i])
        print ('especificidad ',i,": ",especificidad[i])

In [23]:
def sampling_strategy(X,y,n_samples,t='majority'):
    target_classes=''
    if t== 'majority':
        target_classes= y.value_counts() >n_samples
    elif t== 'minority':
        target_classes = y.value_counts() < n_samples
    tc=target_classes[target_classes ==True].index
    #target_classes_all=y.value_counts().index
    sampling_strategy={}
    for target in tc:
        sampling_strategy[target]= n_samples
    return sampling_strategy

In [24]:
def df2Feature_importanceClass(x_train_b,mod,title):
    #shap_test = x_train_b.sample(1000)
    shap_test = x_train_b
    shap_values = shap.TreeExplainer(mod).shap_values(shap_test)
    plt.figure()
    plt.title(title)
    shap.summary_plot(shap_values, shap_test,max_display=30)
    plt.subplots_adjust(left=0.35, right=0.9, top=0.9, bottom=0.3)    
    plt.show();

## Modelo
### Procesamiento

In [25]:
data_ocupacion['mesAnt']=(periodos_x+periodos_y)-diff_month(fechaMaxima,data_ocupacion['FECHA'])
### Ordenar por fecha
data_ocupacion=data_ocupacion.sort_values(["FECHA","COD_NODO"], ascending = (True, True))

### Seleccion Nodos para el Modelo

In [26]:
### Solo Nodos con información completa
seleccion=data_ocupacion[['COD_NODO',variable_prediccion]].groupby('COD_NODO').count()
seleccion[seleccion[variable_prediccion]>=periodos_y+periodos_x+1]
seleccion.rename(columns={variable_prediccion:'noRegVarPre'}, inplace=True)
data_ocupacion=data_ocupacion.merge(seleccion, on='COD_NODO', how='left')
data_ocupacion=data_ocupacion[data_ocupacion['noRegVarPre']>=periodos_y+periodos_x]
data_ocupacion.drop(columns='noRegVarPre', inplace=True)
### Nodos No Segmentados en el periodo de tiempo considerado
data_ocupacion=data_ocupacion[data_ocupacion['SEGMENTADO']!=1]
data_ocupacion.drop(columns='SEGMENTADO', inplace=True)
### Eliminar datos con nullos
data_ocupacion=data_ocupacion.dropna()

###  Selección Columnas

In [27]:
selCols=['COD_NODO','FECHA','OCUPACION_DW','OCUPACION_UP','SERV_TV','SERV_INT','mesAnt']
data_ocupacionC=data_ocupacion.copy()
data_ocupacion=data_ocupacion[selCols]

### Tidy Tables

#### Regresión

In [28]:
cols=data_ocupacion.columns
cols=cols[2:-1]

In [29]:
cols_x=[]
for i, col in enumerate(cols):
    tablaAnalisis = pd.pivot_table(data_ocupacion.reset_index(), index='COD_NODO', columns='mesAnt', values=col)
    tablaAnalisis.rename(columns={x: (col+"-"+str(x)) for x in tablaAnalisis.columns}, inplace=True)
    #Identificacion Columnas X y Y    
    cols_x=cols_x+[(col+"-"+str(x)) for x in range(0,periodos_x)]    
    if i==0:
        tablaAnalisisTotal = tablaAnalisis
    else:
        tablaAnalisisTotal = tablaAnalisisTotal.merge(tablaAnalisis, how='outer', on='COD_NODO')

In [30]:
cols_x=sorted(list(set(cols_x)& set(tablaAnalisisTotal.columns.tolist())))

In [31]:
cols_y=[(variable_prediccion+"-"+str(x)) for x in range(periodos_x,periodos_x+periodos_y-1)]

In [32]:
tablaAnalisisTotal=tablaAnalisisTotal.dropna()

#### Clasificación

In [33]:
colsC=data_ocupacionc.columns
colsC=colsC[2:-1]

NameError: name 'data_ocupacionc' is not defined

In [ ]:
cols_xC=[]
for i, col in enumerate(colsC):
    tablaAnalisis = pd.pivot_table(data_ocupacionC.reset_index(), index='COD_NODO', columns='mesAnt', values=col)
    tablaAnalisis.rename(columns={x: (col+"-"+str(x)) for x in tablaAnalisis.columns}, inplace=True)
    #Identificacion Columnas X y Y    
    cols_xC=cols_xC+[(col+"-"+str(x)) for x in range(0,periodos_x)]    
    if i==0:
        tablaAnalisisTotalC = tablaAnalisis
    else:
        tablaAnalisisTotalC = tablaAnalisisTotalC.merge(tablaAnalisis, how='outer', on='COD_NODO')
tablaAnalisisTotalC=tablaAnalisisTotalC.dropna()

In [ ]:
cols_xC=sorted(list(set(cols_xC)& set(tablaAnalisisTotalC.columns.tolist())))

### Separación Xvar y Yvar (número variable de periodo)

In [ ]:
Xvar=tablaAnalisisTotal[cols_x]
XvarC=tablaAnalisisTotalC[cols_xC]
yvar=tablaAnalisisTotal[cols_y]

In [ ]:
for col in cols_y:    
    yvar[col+'_sem']=yvar[col].apply(semaforoRojo)
    yvar[col+'_sem2']=yvar[col].apply(semaforoNum)    

yColClass=[s + '_sem2' for s in cols_y]

### Separacion Train y Test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(Xvar, yvar, test_size = 0.3, random_state = 86, shuffle= True)

##  Modelos ML

### XGBRegressor

In [ ]:
modelos={}
for yCol in cols_y:
    print(yCol)
    y_train_aux=y_train[yCol]
    y_test_aux=y_test[yCol]
    dicModelo={}        
    dicModelo['modelo']=xgb.XGBRegressor(max_depth=80, verbosity= 0, subsample=0.5,  n_estimators=472, n_jobs=-1, objective='reg:squarederror', booster='gbtree', random_state=42, learning_rate=0.04, reg_lambda=1)
    dicModelo['modelo'].fit(X_train, y_train_aux, eval_set=[(X_train, y_train_aux), (X_test, y_test_aux)])
    dicModelo['predictions'] = dicModelo['modelo'].predict(X_test)
    dicModelo['explanation']=shap.TreeExplainer(dicModelo['modelo'])    
    modelos[yCol]=dicModelo    

####  Variables explicativas

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3,figsize=(40,50))
i=0
for predVar in modelos:
    j=0    
    axes[i//3,i%3].set_title(predVar)    
    xgb.plot_importance(modelos[predVar]['modelo'],ax=axes[i//3,i%3]) 
    j=j+1
    i=i+1

#### Matrices de confusión

##### Semaforo

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3,figsize=(10,5))
i=0
for predVar in modelos:
    y_test_sem = [semaforoVal(x) for x in y_test[predVar]]    
    y_test_p_sem=[semaforoVal(x) for x in modelos[predVar]['predictions']]
    matrizConfucion = confusion_matrix(y_test_sem, y_test_p_sem, labels=['V','A','R'])
    matrizConfucionDF=pd.DataFrame(matrizConfucion, index=['V','A','R'], columns=['V','A','R'])
    sns.heatmap(matrizConfucionDF, annot = True, cmap = 'Greens', fmt='g', ax=axes[i//3,i%3])
    axes[i//3,i%3].set_title(predVar)    
    i=i+1
fig.tight_layout()

#####  Rojos

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3,figsize=(10,5))
i=0
for predVar in modelos:    
    y_test_sem = [semaforoRojo(x) for x in y_test[predVar]]
    y_test_p_sem=[semaforoRojo(x) for x in modelos[predVar]['predictions']]
    matrizConfucion = confusion_matrix(y_test_sem, y_test_p_sem)
    matrizConfucionDF=pd.DataFrame(matrizConfucion)
    sns.heatmap(matrizConfucionDF, annot = True, cmap = 'Greens', fmt='g', ax=axes[i//3,i%3])
    axes[i//3,i%3].set_title(predVar)    
    i=i+1
fig.tight_layout()

##### Curva ROC y AUC

In [ ]:
i=0
for predVar in modelos:
    j=0
    y_test_sem = [semaforoRojo(x) for x in y_test[predVar]]
    y_test_p_sem=[semaforoRojo(x) for x in modelos[predVar]['predictions']]
    fpr1, tpr1, _ = roc_curve(y_test_sem, y_test_p_sem)
    auc1 = roc_auc_score(y_test_sem, y_test_p_sem)
    modelos[predVar]['AUC']=auc1
    plt.plot(fpr1, tpr1, label = predVar+'AUC={}'.format(np.round(auc1, 2)))        
plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1));
plt.title("ROC - Rojos");

##### ROC AMARILLOS

In [ ]:
for predVar in modelos:        
    y_test_sem = [semaforoAmarillo(x) for x in y_test[predVar]]
    y_test_p_sem=[semaforoAmarillo(x) for x in modelos[predVar]['predictions']]
    fpr1, tpr1, _ = roc_curve(y_test_sem, y_test_p_sem)
    auc1 = roc_auc_score(y_test_sem, y_test_p_sem)
    modelos[predVar]['AUC']=auc1
    plt.plot(fpr1, tpr1, label = predVar+'AUC={}'.format(np.round(auc1, 2)))            
plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1));
plt.title("ROC - Amarillos");

##### ROC Rojos y Amarillos

In [ ]:
for predVar in modelos:    
    y_test_sem = [semaforoRojoAmarillo(x) for x in y_test[predVar]]
    y_test_p_sem=[semaforoRojoAmarillo(x) for x in modelos[predVar]['predictions']]
    fpr1, tpr1, _ = roc_curve(y_test_sem, y_test_p_sem)
    auc1 = roc_auc_score(y_test_sem, y_test_p_sem)
    modelos[predVar]['AUC']=auc1
    plt.plot(fpr1, tpr1, label = predVar+'AUC={}'.format(np.round(auc1, 2)))           
plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1));
plt.title("ROC - Amarillos y Rojos");

### LGBMClassifier

#### Agregar Semaforo a variable y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(XvarC, yvar, test_size = 0.3, random_state = 86, shuffle= True)

In [ ]:
modelosClass={}
X_train_Balanceadas={}
for yCol in yColClass:  
    count=y_train[yCol].value_counts()
    n_samples=int(count.max())
    over_sampler = SMOTE(sampling_strategy=sampling_strategy(X_train, y_train[yCol],n_samples,t='minority'),k_neighbors=2)
    X_train_res, y_train_res= over_sampler.fit_resample(X_train,y_train[yCol])
    X_train_Balanceadas[yCol]=X_train_res
    
    print(yCol)
    y_test_aux=y_test[yCol]
    dicModelo={}        
    
    dicModelo['modelo']=LGBMClassifier(n_estimators=500, learning_rate=0.01, reg_alpha=1, reg_lambda=3, objective ='multiclass')
    dicModelo['modelo'].fit(X_train_res, y_train_res) # ajustamos los datos de entrenamiento

    dicModelo['predictions'] =dicModelo['modelo'].predict(X_test)
    dicModelo['explanation']=shap.TreeExplainer(dicModelo['modelo'])
    dicModelo['shap_values']=dicModelo['explanation'].shap_values(XvarC)    
    modelosClass[yCol]=dicModelo

 #### Variables Explicativas

In [ ]:
#Mostrar importancia de variable, aumenta 15 min el tiempo de procesamiento
#for predVar in modelosClass:       
#    print(predVar)    
#    df2Feature_importanceClass(X_train_Balanceadas[predVar],modelosClass[predVar]['modelo'],predVar)  

#### Matrices de Confusión

##### Semaforo

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3,figsize=(10,5))
i=0
for predVar in modelosClass:        
    y_test_sem = y_test[predVar]
    y_test_p_sem=modelosClass[predVar]['predictions']
    matrizConfucion = confusion_matrix(y_test_sem, y_test_p_sem)
    matrizConfucionDF=pd.DataFrame(matrizConfucion, index=['V','A','R'], columns=['V','A','R'])
    print(predVar)
    medidasEvaluacion(matrizConfucion)
    sns.heatmap(matrizConfucionDF, annot = True, cmap = 'Greens', fmt='g', ax=axes[i//3,i%3])
    axes[i//3,i%3].set_title(predVar)    
    i=i+1
fig.tight_layout()

##### Rojos

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3,figsize=(10,4))
i=0
for predVar in modelosClass:  
    print(predVar)
    y_test_sem = [semaforoNum2Rojo(x) for x in y_test[predVar]]
    y_test_p_sem=[semaforoNum2Rojo(x) for x in modelosClass[predVar]['predictions']]
    matrizConfucion = confusion_matrix(y_test_sem, y_test_p_sem)
    matrizConfucionDF=pd.DataFrame(matrizConfucion)
    sns.heatmap(matrizConfucionDF, annot = True, cmap = 'Greens', fmt='g', ax=axes[i//3,i%3])
    axes[i//3,i%3].set_title(predVar)    
    i=i+1
fig.tight_layout()

#####  Curva ROC

##### Solo Rojos

In [ ]:
for predVar in modelosClass:        
    y_test_sem = [semaforoNum2Rojo(x) for x in y_test[predVar]]
    y_test_p_sem=[semaforoNum2Rojo(x) for x in modelosClass[predVar]['predictions']]
    fpr1, tpr1, _ = roc_curve(y_test_sem, y_test_p_sem)
    auc1 = roc_auc_score(y_test_sem, y_test_p_sem)
    modelosClass[predVar]['AUC']=auc1
    plt.plot(fpr1, tpr1, label = predVar+'-'+'AUC={}'.format(np.round(auc1, 2)))            
plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1));

##### Solo Amarillos

In [ ]:
for predVar in modelosClass:        
    y_test_sem = [semaforoNum2Amarillo(x) for x in y_test[predVar]]
    y_test_p_sem=[semaforoNum2Amarillo(x) for x in modelosClass[predVar]['predictions']]
    fpr1, tpr1, _ = roc_curve(y_test_sem, y_test_p_sem)
    auc1 = roc_auc_score(y_test_sem, y_test_p_sem)
    modelosClass[predVar]['AUC']=auc1
    plt.plot(fpr1, tpr1, label = predVar+'-'+'AUC={}'.format(np.round(auc1, 2)))            
plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1));

##### Rojos y amarillos

In [ ]:
for predVar in modelosClass:        
    y_test_sem = [semaforoNum2RojoAmarillo(x) for x in y_test[predVar]]
    y_test_p_sem=[semaforoNum2RojoAmarillo(x) for x in modelosClass[predVar]['predictions']]
    fpr1, tpr1, _ = roc_curve(y_test_sem, y_test_p_sem)
    auc1 = roc_auc_score(y_test_sem, y_test_p_sem)
    modelosClass[predVar]['AUC']=auc1
    plt.plot(fpr1, tpr1, label = predVar+'-'+'AUC={}'.format(np.round(auc1, 2)))            
plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1));

###  NUEVAS PREDICCIONES

In [ ]:
data_ocupacionNp=data_ocupacion.copy()
data_ocupacionNpC=data_ocupacionC.copy()

In [ ]:
periodos_x=6
fechaMaxima=data_ocupacionNp['FECHA'].max()
fechaInicialNp=fechaMaxima + relativedelta(months=-1*periodos_x)

In [ ]:
data_ocupacionNp=data_ocupacionNp[data_ocupacionNp['FECHA']>=fechaInicialNp]
data_ocupacionNpC=data_ocupacionNpC[data_ocupacionNpC['FECHA']>=fechaInicialNp]
data_ocupacionNp['mesAnt']=(periodos_x)-diff_month(fechaMaxima,data_ocupacionNp['FECHA'])
data_ocupacionNpC['mesAnt']=(periodos_x)-diff_month(fechaMaxima,data_ocupacionNpC['FECHA'])

In [ ]:
data_ocupacionNp['FECHA'].max()

In [ ]:
data_ocupacionNp['FECHA'].min()

In [ ]:
colsNp=data_ocupacionNp.columns
colsNp=colsNp[2:-1]
colsNpC=data_ocupacionNpC.columns
colsNpC=colsNpC[2:-1]

In [ ]:
#Tabla para regresión
cols_xNp=[]
for i, col in enumerate(colsNp):
    tablaAnalisisNp = pd.pivot_table(data_ocupacionNp.reset_index(), index='COD_NODO', columns='mesAnt', values=col)
    tablaAnalisisNp.rename(columns={x: (col+"-"+str(x)) for x in tablaAnalisisNp.columns}, inplace=True)    
    cols_xNp=cols_xNp+[(col+"-"+str(x)) for x in range(0,periodos_x+1)]    
    if i==0:
        tablaAnalisisTotalNp = tablaAnalisisNp
    else:
        tablaAnalisisTotalNp = tablaAnalisisTotalNp.merge(tablaAnalisisNp, how='outer', on='COD_NODO')

In [ ]:
#Tabla para clasificación
cols_xNpC=[]
for i, col in enumerate(colsNpC):
    tablaAnalisisNpC = pd.pivot_table(data_ocupacionNpC.reset_index(), index='COD_NODO', columns='mesAnt', values=col)
    tablaAnalisisNpC.rename(columns={x: (col+"-"+str(x)) for x in tablaAnalisisNpC.columns}, inplace=True)    
    cols_xNpC=cols_xNpC+[(col+"-"+str(x)) for x in range(0,periodos_x+1)]    
    if i==0:
        tablaAnalisisTotalNpC = tablaAnalisisNpC
    else:
        tablaAnalisisTotalNpC = tablaAnalisisTotalNpC.merge(tablaAnalisisNpC, how='outer', on='COD_NODO')

In [ ]:
colsNpC

In [ ]:
tablaAnalisisTotalNp=tablaAnalisisTotalNp.dropna()
tablaAnalisisTotalNpC=tablaAnalisisTotalNpC.dropna()

In [ ]:
tablaResultados=tablaAnalisisTotalNp.copy()

#### Resultados XGBOOST 

In [ ]:
for predVar in modelos:
    print(predVar)
    tablaResultados[predVar+'_pR']=modelos[predVar]['modelo'].predict(tablaAnalisisTotalNp)
    tablaResultados[predVar+'_pR_Sem']=tablaResultados[predVar+'_pR'].apply(semaforoVal)

In [ ]:
for predVar in modelosClass:
    tablaResultados[predVar+'_pC']=modelosClass[predVar]['modelo'].predict(tablaAnalisisTotalNpC)
    tablaResultados[predVar+'_pC']=tablaResultados[predVar+'_pC'].apply(semaforoNum2Val)

In [ ]:
tablaResultados

In [ ]:
tablaResultados.to_csv('resultados.csv', index=True,sep=';')